In [1]:
from openqaoa import QAOA
from openqaoa import QUBO
from openqaoa.algorithms import QAOAResult
from openqaoa.backends import create_device

import sys
import os
import asyncio
import json

sys.path.append(os.path.abspath(os.path.join('..', 'functions')))
from JRP import JRP 
from JRPRandomGenerator import JRPRandomGenerator

In [2]:
###INSTANCIA 1
### ESTA INSTANCIA NO TIENE CASOS DONDE CONVIENE CAMBIAR ( (vairables lineales con weight<0)), SU SOLUCION LOGICAMENTE ES UNA CADENA DE TODOS 0

#lower means more priority/affinity
priorities = [1,2,3,4,5]
affinities = [1,2,3,4,5]

agents = list(range(3))
agents_assgJobs = [2,1,0]
#each tuple is form by (agent_index,[affinities of each of the n jobs])
agents_assgJobsAfinnity = [
    [2,3,4]
    ,[5,4,3]
    ,[2,1,3]
]
agents_vacnJobsAfinnity = [
    [5,4,4,3]
    ,[3,2,1,1]
    ,[1,1,1,1]
]


assgJobs = list(range(3))
assgJobs_agents = [2,1,0]
assgJobs_priority = [3,2,5]


vacnJobs = list(range(4))
vacnJobs_priority = [3,2,4,1]

priorityWeightCoeff = 5
affinityWeightCoeff = 5
penalty1 = 5
penalty2 = 5

priorityWeightCoeff = 1
affinityWeightCoeff = 1
penalty1=2
penalty2 = 2

allBinaryVariables = list(range(len(vacnJobs) * len(agents)))

print('assg_jobs: ',assgJobs)
print(vacnJobs)
print(agents)
print(assgJobs_agents)

assg_jobs:  [0, 1, 2]
[0, 1, 2, 3]
[0, 1, 2]
[2, 1, 0]


In [4]:
inst = data = {
            "num_agents": len(agents),
            "num_vacnJobs": len(vacnJobs),
            "priorityWeightCoeff": priorityWeightCoeff,
            "affinityWeightCoeff": affinityWeightCoeff,
            "penalty1": penalty1,
            "penalty2": penalty2,
            "agents": agents,
            "assgJobs": assgJobs,
            "vacnJobs": vacnJobs,
            "agents_assgJobs": agents_assgJobs,
            "agents_assgJobsAfinnity":agents_assgJobsAfinnity,
            "agents_vacnJobsAfinnity":agents_vacnJobsAfinnity,
            "assgJobs_agents": assgJobs_agents,
            "assgJobs_priority": assgJobs_priority,
            "vacnJobs_priority": vacnJobs_priority,
            "allBinaryVariables": allBinaryVariables
        }


In [2]:
jrp_init_configurations = [
    {'num_agents':2 , 'num_vacnJobs':3 , 'control_restrictions':False},
    {'num_agents':4 , 'num_vacnJobs':2 , 'control_restrictions':False}
]

In [8]:
def run_qaoa_workflow(jrp):

    # create the QUBO in openqaoa
    jrp.to_qubo()
    terms,weights = jrp.to_openqaoa_format()
    terms,weights = QUBO.convert_qubo_to_ising(len(jrp.instance_dict['allBinaryVariables']), terms, weights)
    jrp_ising = QUBO(n=len(jrp.instance_dict['allBinaryVariables']),terms=terms,weights=weights)
    
    # create QAOA solver and configurations
    qaoa = QAOA()
    device = create_device(location='local', name='qiskit.shot_simulator')
    qaoa.set_device(device)
    qaoa.set_circuit_properties(p=3, param_type='standard', init_type='ramp', mixer_hamiltonian='x')
    qaoa.set_backend_properties(n_shots=10000)
    qaoa.set_classical_optimizer(method='Powell',maxfev=2, tol=0.001,
                          optimization_progress=True, cost_progress=True, parameter_log=True)
    
    #compile
    qaoa.compile(jrp_ising)

    # get the initial ising gain, using the initial variational parameters
    initial_ising_gain = qaoa.evaluate_circuit(qaoa.variate_params.asdict())['cost']

    # do the QAOA optimization
    qaoa.optimize()
    qaoa_result = qaoa.result
    
    # get the final ising gain, using the optimized variational parameters
    final_ising_gain = qaoa_result.optimized['cost']

    # get 
    qaoa = QAOA()
    device = create_device(location='local', name='qiskit.shot_simulator')
    qaoa.set_device(device)
    qaoa.set_circuit_properties(p=3, param_type='standard', mixer_hamiltonian='x')
    qaoa.set_backend_properties(n_shots=20)
    qaoa.compile(jrp_ising)
    preliminary_qubo_solutions = qaoa.evaluate_circuit(qaoa_result.optimized['angles'])['measurement_results']
    #print(preliminary_qubo_solutions.keys())
    final_qubo_solutions = []
    for solution in preliminary_qubo_solutions.keys():
        if jrp.is_qubo_feasible(solution):
            final_qubo_solutions.append(solution)
    #print(final_qubo_solutions)

    final_standard_gain = None
    for qubo_solution in final_qubo_solutions:
        standard_solution = jrp.quboSolution_to_standardSolution(qubo_solution)
        standard_gain = jrp.calculate_standard_gain(standard_solution)
        
        if final_standard_gain is None or final_standard_gain < standard_gain:
            final_standard_solution = standard_solution
            final_standard_gain = standard_gain
    
    return final_standard_solution,final_standard_gain,initial_ising_gain,final_ising_gain
            

def run_workflow(jrp):   
    initial_standard_solution = [-1 for i in range(jrp.instance_dict['num_agents'])]
    initial_standard_gain = jrp.calculate_standard_gain(initial_standard_solution)
    
    opt_standard_solution,opt_standard_gain = jrp.solve_standard_with_bruteforce(debug_every=0)
   
    final_standard_solution,final_standard_gain,initial_ising_gain,final_ising_gain = run_qaoa_workflow(jrp)
    
#print(qaoa.qaoa_descriptor.abstract_circuit)
sample_workflows(jrp_init_configurations[1],1)


In [4]:
#a = [[0, 1], [0, 2], [1, 2], [3, 4], [3, 5], [4, 5], [6, 7], [8, 6], [8, 7], [9, 10], [9, 11], [10, 11], [0, 3], [0, 6], [0, 9], [3, 6], [9, 3], [9, 6], [1, 4], [1, 7], [1, 10], [4, 7], [10, 4], [10, 7], [2, 5], [8, 2], [2, 11], [8, 5], [11, 5], [8, 11], [0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11]]
#b= [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [0, 1], [0, 2], [1, 2], [3, 4], [3, 5], [4, 5], [6, 7], [8, 6], [8, 7], [9, 10], [9, 11], [10, 11], [0, 3], [0, 6], [0, 9], [3, 6], [9, 3], [9, 6], [1, 4], [1, 7], [1, 10], [4, 7], [10, 4], [10, 7], [2, 5], [8, 2], [2, 11], [8, 5], [11, 5], [8, 11]]
#a

#print(a == b)

In [3]:
def sample_workflows(jrp_init_configuration,sample_size):
    jrp_gen = JRPRandomGenerator(**jrp_init_configuration)
    
    for sample in range(sample_size):
        jrp_instance_dict = jrp_gen.generate_random_instance()
        jrp_instance_dict = json.loads(jrp_instance_dict)
        #print(jrp_instance_dict)
        jrp = JRP(jrp_instance_dict)
        run_workflow(jrp)
